In [0]:
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split
import numpy as np
import os

In [1]:
! git clone https://github.com/raghunanden/image-classifiction-framework.git

Cloning into 'image-classifiction-framework'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 2), reused 22 (delta 2), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [2]:
from google.colab import auth
auth.authenticate_user()
project_id = 'vocal-facet-272514'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey

gs://raghukaggledata/


In [3]:
bucket_name = 'raghukaggledata'
!gsutil -m cp -r gs://{bucket_name}/aptos2019-blindness-detection.zip /content/image-classifiction-framework/dataset/

Copying gs://raghukaggledata/aptos2019-blindness-detection.zip...
/ [1/1 files][  9.5 GiB/  9.5 GiB] 100% Done  53.8 MiB/s ETA 00:00:00           
Operation completed over 1 objects/9.5 GiB.                                      


In [0]:
## read_data_csv
## split it into train test and validation
## organize the images accordingly
## build input pipeline using tf.data

In [0]:
%%bash
cd /content/image-classifiction-framework/dataset/
unzip -qq aptos2019-blindness-detection.zip

In [0]:
train_df = pd.read_csv("/content/image-classifiction-framework/dataset/train.csv")

In [29]:
train_df.head(1)

,id_code,diagnosis
0,000c1434d8d7,2


In [0]:
X_train, X_eval, y_train, y_eval = train_test_split(
    train_df["id_code"], train_df["diagnosis"], test_size=0.25, random_state=42)
train = pd.DataFrame()
train["id_code"] = X_train
train["diagnosis"] = y_train
validation = pd.DataFrame()
validation["id_code"] = X_eval
validation["diagnosis"] = y_eval

In [0]:
def move_file(file_name, folder=None):
  # Source path

  source = '/content/image-classifiction-framework/dataset/train_images/{}.png'.format(file_name)
    
  # Destination path  
  destination = '/content/image-classifiction-framework/dataset/{}/{}.png'.format(folder,file_name)
    
  # Move the content of  
  # source to destination  
  dest = shutil.move(source, destination)  

In [39]:
if not os.path.exists('/content/image-classifiction-framework/dataset/train'):
    os.makedirs('/content/image-classifiction-framework/dataset/train')
if not os.path.exists('/content/image-classifiction-framework/dataset/validation'):
    os.makedirs('/content/image-classifiction-framework/dataset/validation')
train["id_code"].apply(lambda x: move_file(x, folder='train'))
validation["id_code"].apply(lambda x: move_file(x, folder='validation'))

2059    None
1105    None
818     None
1237    None
463     None
        ... 
1075    None
1108    None
2877    None
1798    None
2177    None
Name: id_code, Length: 916, dtype: object